In [ ]:
#For scraping
import requests
import pandas as pd
from bs4 import BeautifulSoup
import html5lib
import re
from datetime import datetime

#For geojson
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt
from mpl_toolkits.axes_grid1.axes_divider import make_axes_locatable
import geopandas as gpd
import pandas as pd
import math
import calendar

## Daily scrape of LA County Public Health table

In [ ]:
#Chose what to scrape
archive = False
titlestring='1206'
stringprint='12/06/20'

if archive:     
    url = 'data/'+titlestring+'.html' #Uses locally downloaded file from wayback machine to lookup archived site
    soup = BeautifulSoup(open(url),"html.parser")
else:
    url ='http://publichealth.lacounty.gov/media/Coronavirus/locations.htm'
    response = requests.get(url)
    soup = BeautifulSoup(response.text, "html.parser")
#    datetime.today().strftime('%Y-%m-%d')
#    titlestring=datetime.today().strftime('%m%d') #Use today's date

In [ ]:
liout = soup.find_all('table', {'class' : 'overflow-y'})
alltables = pd.read_html(str(liout))

In [ ]:
df = alltables[0]

In [ ]:
df.iloc[:,0] = df.iloc[:,0].str.replace('--','suppressed')
#df.iloc[:,0] = df.iloc[:,0].str.replace('City of ','')
#df.iloc[:,0] = df.iloc[:,0].str.replace('Unincorporated - ','')
#df.iloc[:,0] = df.iloc[:,0].str.replace('Los Angeles - ','')
df.iloc[:,0] = df.iloc[:,0].str.replace('*','')
#df.iloc[:,0] = df.iloc[:,0].str.replace('- ','')

In [ ]:
#Add two blank rows for Total and Pasadena
df.loc[len(df)]=''
df.loc[len(df)]=''

In [ ]:
pd.set_option('display.max_rows', None)
df

In [ ]:
#liout2 = soup.find_all('table', {'class' : 'table table-striped table-bordered table-sm'})[0].find_all('td')
#summary = pd.read_html(soup)

In [ ]:
df.rename(columns={"CITY/COMMUNITY**": "Locations"}, inplace=True)

In [ ]:
#Add total - currently hardcoded
column_marker =0 
for column in soup.find_all('thead', {'class' : 'bg-dark'})[1].find_all('td'): 

    if (column_marker==0):
        print('Renaming ', column.get_text(), 'to Total')
        df.iat[-1, column_marker] = 'Total'
    else:
        df.iat[-1, column_marker] = column.get_text()
        print(column.get_text())
    column_marker += 1

#df.iat[-1, 3] = int(soup.find_all('tr')[11].find_all('td')[1].get_text())
    

In [ ]:
soup.find_all('tr')[6]

In [ ]:
#Add Pasadena back in 
column_marker =0
pasadenapop = 141371.
for column in soup.find_all('tr')[11].find_all('td'):
    print(column.get_text())

In [ ]:
#Add Pasadena back in 
#column_marker =0
pasadenapop = 141371.
#for column in soup.find_all('tr')[8].find_all('td'): #Previously 4, change as of May ~15
#    if(column_marker <2):
#        print(column.get_text())
#        df.iat[-3, column_marker] = column.get_text().replace('-','').replace(' ','')
#        if(column_marker ==1):
#            df.iat[-3, column_marker+1] = int( column.get_text().replace('-','')) /pasadenapop *100000
#    else:
#        print('Skipping:', column.get_text())
#    column_marker += 1
df.iat[-3, 0] = 'City of Pasadena'#soup.find_all('tr')[7].find_all('td')[0].get_text().replace('-','').replace(' ','')
df.iat[-3, 1] = int(soup.find_all('tr')[7].find_all('td')[1].get_text())
df.iat[-3, 2] = int(soup.find_all('tr')[7].find_all('td')[1].get_text())/pasadenapop *100000
df.iat[-3, 3] = int(soup.find_all('tr')[11].find_all('td')[1].get_text())
df.iat[-3, 4] = int(soup.find_all('tr')[11].find_all('td')[1].get_text())/pasadenapop *100000    

In [ ]:
#Add Long Beach back in (No longer in table list like it was before)
#df.at['Long Beach',colnames[1]] = soup.find_all('tr')[7].find_all('td')[1].text.replace('*','') #Previously 3, not 7
#df.at['Long Beach',colnames[2]] = int(soup.find_all('tr')[7].find_all('td')[1].text)/longbeachpop*100000

#column_marker =0
longbeachpop = 467354.
#for column in soup.find_all('tr')[7].find_all('td'): #Previously 4, change as of May ~15
#    if(column_marker <2):
#        print(column.get_text())
#        df.iat[-2, column_marker] = column.get_text().replace('- ','')
#        if(column_marker ==1):
#            df.iat[-2, column_marker+1] = int( column.get_text().replace('- ','')) /longbeachpop *100000
#    else:
#        print('Skipping:', column.get_text())
#    column_marker += 1
df.iat[-2, 0] = 'City of Long Beach'
df.iat[-2, 1] = int(soup.find_all('tr')[6].find_all('td')[1].get_text())
df.iat[-2, 2] = int(soup.find_all('tr')[6].find_all('td')[1].get_text())/longbeachpop *100000
df.iat[-2, 3] = int(soup.find_all('tr')[10].find_all('td')[1].get_text())
df.iat[-2, 4] = int(soup.find_all('tr')[10].find_all('td')[1].get_text())/longbeachpop *100000

In [ ]:
soup.find_all('tr')[7].find_all('td')[1].get_text()

In [ ]:
df

In [ ]:
#Other cleanup - this is different once the index has been changed:
df.loc[df['Locations'].str.contains('Under Investigation'), 'Locations'] = 'Under Investigation'
#df.loc[df['Locations'].str.match('Los Angeles'), 'Locations'] = 'Los Angeles - AGGREGATE'
df.loc[df['Locations'].str.match('Laboratory Confirmed Cases (LCC)'), 'Locations'] = 'Total'

df[df['Locations'].str.contains('AGGREGATE')]

In [ ]:
df.set_index('Locations', inplace=True)
df

#Fix Long Beach and Pasadena
column_marker = 0
for column in soup.find_all('tr')[3].find_all('td'):
    print(df.loc['Long Beach'])
    df.at['Long Beach', colnames[column_marker]] = column.get_text()
    #    df.iat[df.loc['Long Beach'] , column_marker] = column.get_text()
    #    print('Before', df[df['Locations'].str.contains('Long Beach')])
    #    df.iat[df.index(df['Locations'].str.contains('Long Beach')).tolist(), column_marker] = column.get_text()
    column_marker += 1
    #    print('After', df[df['Locations'].str.contains('Long Beach')])
    

In [ ]:
df.loc['City of Long Beach']

In [ ]:
#Write out to CSV. Make the format match what covid_la.ipynb expects
df.index.names=['city']
df.rename(columns={'Locations': 'city', 'Total Cases': 'count', 'Rate': 'rate'}).to_csv(
    "./data/covid_bos_"+titlestring+".csv",
    index=True,
    encoding="utf-8"
)
#THERE IS A KNOWN BUG that Under Investigation is dropped in archived scrapes! Must be readded

In [ ]:
#Read in LA neighborhood map
nbd= gpd.read_file("data/bos_stat_areas.geojson")

#Drop islands
nbd=nbd[~nbd.label.str.contains('Catalina')]
nbd=nbd[~nbd.label.str.contains('Clemente')]

In [ ]:
# create a dictionary 
# key = old name 
# value = new name 


dict = {'Cases':'Confirmed Cases as of '+stringprint,
        'Case Rate1':'Infection rate adjusted for population (#/100,000)',
        'Deaths':'Attributed Deaths',
        'Death Rate2':'Death rate adjusted for population (#/100,000)'} 
  
# call rename () method 
df.rename(columns=dict, 
          inplace=True) 

In [ ]:
#df.drop('Unincorporated - Santa Catalina Island', axis=0)
#df.drop('Unincorporated - San Clemente Island', axis=0)
mymerged = nbd.set_index('label').join(df)
mymerged.to_file("./data/combined_reports_reg_bos_"+titlestring+".json",index=True, driver="GeoJSON")